In [14]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
def get_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

In [16]:
def calculate_moving_averages(data, short_window, long_window):
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()
    return data

In [17]:
def crossover(data):
    buy_signals = []
    sell_signals = []
    position = -1 #-1 no position, 1 long position

    for i in range (len(data)):
        if data['Short_MA'][i] > data['Long_MA'][i] and position != 1:
            buy_signals.append(data['Close'][i])
        elif data['Short_MA'][i] < data['Long_MA'][i]:
            sell_signals.append(data['Close'][i])
            position = 0 #sold position
        
    return buy_signals, sell_signals

In [18]:
def plot_data(data):
    plt.figure(figsize=(12,6))
    plt.plot(data['Close'], label='Close Price', alpha=0.35)
    plt.plot(data['Short_MA'], label=f'Short {short_window}-Day MA', alpha=0.75)
    plt.plot(data['Long_MA'], label=f'Long {long_window}-Day MA', alpha=0.75)
    
    plt.scatter(data.index, data['Buy'], label='Buy Signal', marker='^', color='green', alpha=1)
    plt.scatter(data.index, data['Sell'], label='Sell Signal', marker='v', color='red', alpha=1)
    
    plt.title(f'Moving Average Crossover Strategy for {symbol}')
    plt.legend(loc='best')
    plt.show()

In [19]:
if __name__ == "__main__":
    # User-defined parameters
    symbol = 'AAPL'  # Stock ticker symbol (you can change it to any stock symbol)
    start_date = '2020-01-01'
    end_date = '2023-01-01'
    short_window = 10  # Short-term moving average window
    long_window = 50   # Long-term moving average window

    # Step 1: Fetch data
    data = get_data(symbol, start_date, end_date)
    
    # Step 2: Calculate moving averages
    data = calculate_moving_averages(data, short_window, long_window)
    
    # Step 3: Generate buy/sell signals
    data['Buy'], data['Sell'] = crossover(data)
    
    # Step 4: Plot the data with buy/sell signals
    plot_data(data)

[*********************100%***********************]  1 of 1 completed
C:\Users\Admin\AppData\Local\Temp\ipykernel_1992\1604333662.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data['Short_MA'][i] > data['Long_MA'][i] and position != 1:
C:\Users\Admin\AppData\Local\Temp\ipykernel_1992\1604333662.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif data['Short_MA'][i] < data['Long_MA'][i]:
C:\Users\Admin\AppData\Local\Temp\ipykernel_1992\1604333662.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent

ValueError: Length of values (410) does not match length of index (756)